In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise.model_selection import KFold
from surprise.model_selection.validation import cross_validate
from sklearn.feature_extraction.text import CountVectorizer
from surprise import SVD,Reader,Dataset
from ast import literal_eval


#function to convert the datatype into int
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan
    
    
#function to split the strings separated by ','
def listing(x):
    try: 
        return x.split(",")
    except:
        return np.nan

    
#loading the dataframe from all the datasets
datadf = pd.read_csv('../dataset/movies_metadata.csv')
linkdf = pd.read_csv('../dataset/links_small.csv')
ratedf = pd.read_csv('../dataset/ratings_small.csv')
creddf = pd.read_csv('../dataset/credits.csv')
keydf = pd.read_csv('../dataset/keywords.csv')


datadf['overview']= datadf['overview'].fillna('')
datadf['overview']= datadf['overview'].apply(listing)

#removing all the entries with duplicate ids
keydf.drop_duplicates(subset ="id", keep ='first',inplace = True)
creddf.drop_duplicates(subset ="id", keep ='first',inplace = True)

#removing all the entries with duplicate title
datadf.drop_duplicates(subset ="title", keep ='first',inplace = True)

#converting the datatype to int
keydf['id'] = keydf['id'].astype('int')
creddf['id'] = creddf['id'].astype('int')

datadf['id']= datadf['id'].apply(clean_ids)

#remving entries with null id
datadf = datadf[datadf['id'].notnull()]

datadf['id']=datadf['id'].astype('int')
linkdf['tmdbId']=linkdf['tmdbId'].apply(clean_ids)
linkdf = linkdf[linkdf['tmdbId'].notnull()]
linkdf['tmdbId']=linkdf['tmdbId'].astype('int')

#creating a new column tmdbId equal to id
datadf['tmdbId']=datadf['id']
datadf=datadf.merge(linkdf,on='tmdbId')

#merging datadf with creddf and keydf
datadf = datadf.merge(creddf, on='id')
datadf = datadf.merge(keydf, on='id')

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    datadf[feature] = datadf[feature].apply(literal_eval)
    
#extracting the director's name
def crewdirector(x):
    for creww in x:
        if creww['job'] == 'Director':
            return creww['name']
    return np.nan
datadf['director'] = datadf['crew'].apply(crewdirector)

#extracting top three names
def generate_list(x):
    if isinstance(x, list):
        names = [key['name'] for key in x]
        if len(names) > 2:
            names = names[:2]
        return names

    return []

datadf['cast'] = datadf['cast'].apply(generate_list)
datadf['keywords'] = datadf['keywords'].apply(generate_list)
datadf['genres'] = datadf['genres'].apply(lambda x: [i['name'] for i in x] if
isinstance(x, list) else [])
datadf['genres'] = datadf['genres'].apply(lambda x: x[:3])



def cleanspaces(x):
     if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
     else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
for feature in ['cast', 'director', 'genres', 'keywords']:
     datadf[feature] = datadf[feature].apply(cleanspaces)
        
#merge the different features
def mergee(x):
     return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast'])+ ' ' +x['director'] + ' ' + ' '.join(x['genres'])+' '+' '.join(x['overview'])

#creating new column merged
datadf['merged'] = datadf.apply(mergee, axis=1)


#making the similarity matrix
tfidf = TfidfVectorizer(stop_words='english')
matrix = tfidf.fit_transform(datadf['merged'])
simmatrix = linear_kernel(matrix, matrix)

datadf = datadf.reset_index()
indices = pd.Series(datadf.index, index=datadf['title'])

#making the dataset of smaller movies
datadf.to_csv('../dataset/movies_small.csv',index=False)

#building SVD based collaborative filter
reader = Reader() 
data = Dataset.load_from_df(ratedf[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(data)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])
trainset = data.build_full_trainset()
svd.fit(trainset)

links2df = pd.read_csv('../dataset/links_small.csv')
links2df['tmdbId']=links2df['tmdbId'].apply(clean_ids)
links2df = links2df[links2df['tmdbId'].notnull()]
links2df['tmdbId']=links2df['tmdbId'].astype('int')

links2df = links2df.merge(datadf[['title', 'tmdbId']], on='tmdbId')

#title to tmdbId and tmdbId to title mappings
links2df = links2df.set_index('title')
links3df=   links2df.set_index('tmdbId')

#making the final recommendation function
def recfunction(user_id, title):
    indx = indices[title]
    tmdbId = links2df.loc[title]['tmdbId']
    sim_scores = list(enumerate(simmatrix[indx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:25]
    movie_indices = [i[0] for i in sim_scores]  
    pred_movies = datadf.iloc[movie_indices][['title', 'runtime','vote_average', 'tmdbId']] 
    pred_movies['est'] = pred_movies['tmdbId'].apply(lambda x: svd.predict(user_id, links3df.loc[x]['movieId']).est)   
    pred_movies = pred_movies.sort_values('est', ascending=False) 
    pred_movies.columns = ['Title', 'Runtime','Vote Average', 'TMDb Id', 'Estimated Prediction']
    return pred_movies.head(10)


C:\Users\Admin\AppData\Local\Temp\ipykernel_20932\2688569747.py:30: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  datadf = pd.read_csv('../dataset/movies_metadata.csv')


In [5]:
recfunction(2,'Kung Fu Hustle')

,Title,Runtime,Vote Average,TMDb Id,Estimated Prediction
2315,Drunken Master,111.0,7.2,11230,4.023086
1500,West Side Story,152.0,7.0,1725,3.901027
6740,Kung Fu Panda,90.0,6.9,9502,3.850734
2276,Armour of God II: Operation Condor,106.0,7.0,10975,3.825850
6438,The 36th Chamber of Shaolin,115.0,7.6,11841,3.807835
3857,Dragon: The Bruce Lee Story,120.0,6.8,10423,3.804782
5148,The Legend I,106.0,7.1,17653,3.762561
5724,Fist of Legend,103.0,7.4,17809,3.630422
2968,The Kentucky Fried Movie,83.0,6.4,11598,3.579950
8184,Particle Fever,99.0,7.1,202141,3.545539


In [6]:
recfunction(2, 'The Conjuring')

,Title,Runtime,Vote Average,TMDb Id,Estimated Prediction
1820,Happiness,139.0,7.4,10683,4.112545
6650,[REC],78.0,7.1,8329,3.774033
3735,Things Behind the Sun,120.0,5.9,102933,3.716733
8370,The Borderlands,89.0,5.1,207774,3.658163
5938,Electra Glide in Blue,114.0,6.6,26332,3.609974
3037,The Spiral Staircase,83.0,6.6,27452,3.587411
8672,The Conjuring 2,134.0,7.0,259693,3.569605
7491,Henry's Crime,108.0,5.8,53172,3.526260
5711,Shining Through,132.0,6.2,31962,3.499536
5461,The Boston Strangler,116.0,6.7,26690,3.498059


In [8]:
import pickle
pickle.dump(datadf,open('movies.pkl','wb'))

In [9]:
userdf = ratedf

In [10]:
userdf.drop_duplicates(subset ="userId", keep ='first',inplace = True)


In [11]:
pickle.dump(userdf,open('users.pkl','wb'))

In [12]:
pickle.dump(links2df,open('links.pkl','wb'))

In [13]:
pickle.dump(links3df,open('links1.pkl','wb'))

In [14]:
pickle.dump(indices,open('indices.pkl','wb'))

In [15]:
pickle.dump(simmatrix,open('matrix.pkl','wb'))

In [16]:
pickle.dump(svd,open('svd1.pkl','wb'))